# To clean the data a bit

First, remove the columns we don't need, and also the data points where 0 is traded

In [5]:
import pandas as pd
import numpy as np
import json

In [2]:
data = pd.read_csv("EU_trade_electricity.csv")

data.head()

,DATAFLOW,LAST UPDATE,freq,siec,partner,unit,geo,TIME_PERIOD,OBS_VALUE,OBS_FLAG,CONF_STATUS
0,ESTAT:NRG_TI_EH(1.0),24/04/25 23:00:00,Annual,Electricity,Austria,Gigawatt-hour,Albania,2018,0.0,NaN,NaN
1,ESTAT:NRG_TI_EH(1.0),24/04/25 23:00:00,Annual,Electricity,Austria,Gigawatt-hour,Albania,2019,0.0,NaN,NaN
2,ESTAT:NRG_TI_EH(1.0),24/04/25 23:00:00,Annual,Electricity,Austria,Gigawatt-hour,Albania,2020,0.0,NaN,NaN
3,ESTAT:NRG_TI_EH(1.0),24/04/25 23:00:00,Annual,Electricity,Austria,Gigawatt-hour,Albania,2021,0.0,NaN,NaN
4,ESTAT:NRG_TI_EH(1.0),24/04/25 23:00:00,Annual,Electricity,Austria,Gigawatt-hour,Albania,2022,0.0,NaN,NaN


In [ ]:

data = data.filter(items=["partner", "geo", "TIME_PERIOD", "OBS_VALUE"])
data = data[data["OBS_VALUE"] > 0]
data.head()

,partner,geo,TIME_PERIOD,OBS_VALUE
36,Austria,Czechia,2018,112.561
37,Austria,Czechia,2019,144.762
38,Austria,Czechia,2020,378.420
39,Austria,Czechia,2021,197.000
40,Austria,Czechia,2022,37.188


In [10]:
data.to_csv("cleaned_EU_trade.csv", index=False)

## Now cleaning the big electricity production dataset

In [28]:
electricity = pd.read_csv("country-energy-raw.csv", header=None)
electricity.head()

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,NaN,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
1,Germany,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Nuclear,161.126,162.74,156.6,156.807,158.712,154.612,158.71,133.209,140.886,...,91.8,86.765,80.038,72.155,71.866,70.992,60.914,65.4413,31.892,6.7229241
3,Fossil fuels,361.442,364.835,365.426,388.591,383.051,383.747,386.138,397.892,385.399,...,342.4846,343.0874,353.6123,340.0581,321.3662,274.929,244.3158,274.6684,278.7447597,263.0143562
4,Coal,296.687,293.734,299.594,304.627,298.76,288.142,288.935,297.106,275.208,...,264.0556,262.3514,254.0213,235.2061,221.9372,169.043,133.9115,162.8818,176.7482382,127.773


['2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023']


In [ ]:
import csv
import json
from collections import defaultdict

# I definitely Chatted this
def safe_float(x):
    try:
        return float(x)
    except (ValueError, TypeError):
        return None

# Read CSV file
data = []
with open('country-energy-raw.csv', newline='') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        data.append(row)

# Extract header (years)
years = data[0][1:]

result = defaultdict(dict)

current_country = None
parent_category = None

for row in data[1:]:
    if not any(row[1:]):  # Country name row
        current_country = row[0].strip()
        continue

    # Determine indent level by leading spaces
    indent_level = len(row[0]) - len(row[0].lstrip())
    category_name = row[0].strip()

    # Use the category name directly as the key
    if indent_level == 0:  # Top-level category (e.g., "Nuclear", "Fossil fuels")
        key_name = category_name
        parent_category = category_name
    elif indent_level == 4:  # Subcategory (e.g., "Coal", "Natural gas")
        key_name = category_name
    elif indent_level == 8:  # Sub-subcategory (e.g., "Solar", "Wind")
        key_name = category_name
    else:
        key_name = category_name

    # Convert values safely
    values = [safe_float(x) for x in row[1:]]

    # Store in result
    result[current_country][key_name] = dict(zip(years, values))

# Save as JSON
with open('country-energy-cleaned.json', 'w') as jsonfile:
    json.dump(result, jsonfile, indent=2)
